In [1]:
import requests
import os
import sqlite3
import pandas as pd

Pull data from SpaceX API

In [2]:
# Pick a dataset and the appropriate url (will require url+"<data>" in other calls to pull specific data)
url = "https://api.spacexdata.com/v4/"
# Get launch data
launches = requests.get(url+"launches").json()
# Get payload data
payloads = requests.get(url+"payloads").json()
# Get rockets data
rockets = requests.get(url+"rockets").json()

In [3]:
print(f"There are {len(launches)} launches")
print(f"There are {len(payloads)} payloads")
print(f"There are {len(rockets)} rockets")

There are 205 launches
There are 225 payloads
There are 4 rockets


Clean SpaceX returned data

In [4]:
# Specify columns that we want to keep from the raw data pull
launch_cols = ['id', 'name', 'date_utc', 'rocket']
# Break down nested JSON struct
df_launches = pd.json_normalize(launches)
# Where we keep columns specified
df_launches = df_launches[launch_cols]
# Clean date to the day
df_launches['date_utc'] = pd.to_datetime(df_launches['date_utc']).dt.strftime('%Y-%m-%d')
# Check for # Check for missing rocket data (will fail pipeline)
assert df_launches['rocket'].notnull().all(), "Some launches are missing rocket info!"
# Rename column
df_launches.rename(columns={'rocket': 'rocket_id'}, inplace=True)
# display(df_launches)

Remove the database if exists then create the spacex.db for ingesting into silver tables

In [5]:
# Delete the old database file if it exists
if os.path.exists("spacex.db"):
    os.remove("spacex.db")
    print("spacex.db database deleted")

spacex.db database deleted


In [6]:
# Create a connection (this creates spacex.db in your current Colab environment)
conn = sqlite3.connect("spacex.db")

# Create a cursor
cur = conn.cursor()

schema = """
CREATE TABLE IF NOT EXISTS rockets (
    id TEXT PRIMARY KEY,
    name TEXT,
    type TEXT,
    cost_per_launch INTEGER

);

CREATE TABLE IF NOT EXISTS launches (
    id TEXT PRIMARY KEY,
    name TEXT,
    date_utc TEXT,
    rocket_id TEXT
);

CREATE TABLE IF NOT EXISTS payloads (
    id TEXT PRIMARY KEY,
    name TEXT,
    type TEXT,
    mass_kg INTEGER,
    launch_id TEXT
);
"""

cur.executescript(schema)
conn.commit()
print("Database schema created")

Database schema created


In [7]:
# Create index
cur.execute("CREATE INDEX IF NOT EXISTS idx_rocket_id ON launches(rocket_id);")
cur.execute("CREATE INDEX IF NOT EXISTS idx_launch_id ON payloads(launch_id);")
conn.commit()

# Check index
# cur.execute("PRAGMA index_list('launches')")
# print(cur.fetchall())

Insert data into appropriate tables (with pandas df)

In [8]:
# Insert data from df_launches into launches table
# if_exists to prevent duplicate records
df_launches.to_sql('launches', conn, if_exists='append', index=False)

205

Insert data into appropriate tables (loop through python)

In [9]:
# Insert data from api into rockets table, with only the columns we want
# ignore preventing pk conflicts
for r in rockets:
    cur.execute("""
        INSERT OR IGNORE INTO rockets
        (id, name, type, cost_per_launch)
        VALUES (?, ?, ?, ?)
    """, (
        r["id"], r["name"], r["type"], r["cost_per_launch"]
    ))

conn.commit()
print("Rockets data inserted")

Rockets data inserted


In [10]:
# Insert data from api into payloads table, with only the columns we want
# ignore preventing pk conflicts
for p in payloads:
    cur.execute("""
        INSERT OR IGNORE INTO payloads
        (id, name, type, mass_kg, launch_id)
        VALUES (?, ?, ?, ?, ?)
    """, (
        p["id"], p["name"], p["type"], p["mass_kg"], p["launch"]
    ))

conn.commit()
print("Payloads data inserted")

Payloads data inserted


Checking database size

In [11]:
db_path = "spacex.db"
size_bytes = os.path.getsize(db_path)
size_mb = size_bytes / (1024 * 1024)

print(f"Database size: {size_mb:.2f} MB")

Database size: 0.11 MB


SQL analysis

In [12]:
# Average payload weight per launch type
df_1 = pd.read_sql_query('''
SELECT p.type, avg(p.mass_kg) as avg_weight
FROM payloads p
JOIN launches l ON p.launch_id = l.id
JOIN rockets r ON l.rocket_id = r.id
GROUP BY p.type
HAVING avg_weight IS NOT null
ORDER BY avg_weight DESC
''', conn)
df_1

,type,avg_weight
0,Crew Dragon,11434.333333
1,Satellite,7459.639329
2,Dragon 2.0,3150.000000
3,Dragon 1.1,2578.761111
4,Lander,585.000000
5,Dragon 1.0,500.500000


In [13]:
# Which rocket has cost spacex the most to launch over its lifetime
df_2 = pd.read_sql_query('''
WITH launch_count AS (
  SELECT count(*) as num_launch, rocket_id
  FROM launches
  GROUP BY rocket_id
)

SELECT r.name, (lc.num_launch*r.cost_per_launch) as total_launch_cost
FROM launch_count lc
LEFT JOIN rockets r
ON lc.rocket_id = r.id
ORDER BY total_launch_cost DESC
''', conn)
df_2

,name,total_launch_cost
0,Falcon 9,9750000000
1,Falcon Heavy,450000000
2,Falcon 1,33500000


Pandas Analysis

In [14]:
# Load tables
rockets_df = pd.read_sql_query("SELECT * FROM rockets", conn)
launches_df = pd.read_sql_query("SELECT * FROM launches", conn)
payloads_df = pd.read_sql_query("SELECT * FROM payloads", conn)


# Merge payloads, launches, rockets into one df
df = payloads_df.merge(launches_df, left_on='launch_id', right_on='id', suffixes=('_payload', '_launch'))
df = df.merge(rockets_df, left_on='rocket_id', right_on='id', suffixes=('', '_rocket'))

In [15]:
# Show number of launches per year using Pandas
df['date_utc'] = pd.to_datetime(df['date_utc'])
launches_per_year = df.groupby(df['date_utc'].dt.year)['launch_id'].nunique().reset_index()
launches_per_year = launches_per_year.rename(columns={'launch_id': 'num_launches'})
launches_per_year

,date_utc,num_launches
0,2006,1
1,2007,1
2,2008,2
3,2009,1
4,2010,2
5,2012,2
6,2013,3
7,2014,6
8,2015,7
9,2016,9
